In [384]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm

In [385]:
def load_original_data():
    return pd.read_csv("Data_Marketing_Customer_Analysis_Round2.csv" )

In [386]:
ca_df=load_original_data()

### Data cleaning

In [387]:
# dropping the irrelevant columns
ca_df.drop(columns=['Unnamed: 0', 'Customer', 'Effective To Date', 'Policy Type', 'Vehicle Type'], inplace=True)

In [388]:
# filtering out 0 Income
ca_df = ca_df[ca_df['Income']>0]

In [389]:
# categorizing Policy
def policy_category(x):
    if x[0] == 'P':
        return 'Personal'
    elif x[0] == 'C':
        return 'Corporate'
    elif x[0] == 'S':
        return 'Special'
    
ca_df['Policy'] = list(map(policy_category, ca_df['Policy']))

In [390]:
# label encoding for Coverage
def coverage_label(x):
    if x == 'Basic':
        return 0
    elif x == 'Extended':
        return 1
    elif x == 'Premium':
        return 2

ca_df['Coverage Transformed'] = list(map(coverage_label, ca_df['Coverage']))

In [391]:
# replacing the Coverage column
ca_df.drop(columns=['Coverage'], inplace=True)

In [392]:
# dropping null values
ca_df.dropna(inplace=True)

In [393]:
# resetting index
ca_df.reset_index(inplace=True)

In [394]:
ca_df.drop(columns=['index'], inplace=True)
ca_df

,State,Customer Lifetime Value,Response,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Coverage Transformed
0,Arizona,4809.216960,No,College,Employed,M,48029,Suburban,Married,61,7.0,52,0.0,9,Corporate,Offer3,Agent,292.800000,Four-Door Car,Medsize,0
1,Washington,14947.917300,No,Bachelor,Employed,M,22139,Suburban,Single,100,34.0,31,0.0,2,Personal,Offer3,Call Center,480.000000,SUV,Medsize,0
2,Oregon,22332.439460,Yes,College,Employed,M,49078,Suburban,Single,97,10.0,3,0.0,2,Corporate,Offer2,Branch,484.013411,Four-Door Car,Medsize,1
3,California,5035.035257,No,Doctor,Employed,F,37405,Urban,Married,63,8.0,99,3.0,4,Corporate,Offer2,Branch,287.556107,Four-Door Car,Medsize,0
4,California,4956.247235,No,High School or Below,Employed,M,87197,Urban,Single,63,35.0,45,0.0,3,Personal,Offer2,Branch,247.402353,Two-Door Car,Medsize,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6812,California,3579.023825,No,High School or Below,Employed,F,28304,Suburban,Married,91,10.0,30,2.0,1,Personal,Offer2,Agent,655.200000,Four-Door Car,Medsize,1
6813,Arizona,2771.663013,No,College,Employed,M,59855,Suburban,Married,74,30.0,82,4.0,1,Personal,Offer2,Branch,355.200000,Two-Door Car,Medsize,0
6814,Oregon,5259.444853,No,College,Employed,F,61146,Urban,Married,65,7.0,68,0.0,6,Personal,Offer2,Branch,273.018929,Four-Door Car,Medsize,0
6815,Arizona,23893.304100,No,Bachelor,Employed,F,39837,Rural,Married,201,11.0,63,0.0,2,Corporate,Offer1,Web,381.306996,Luxury SUV,Medsize,1


### creating X, y split

In [395]:
X = ca_df.drop('Total Claim Amount', axis=1)
y = ca_df['Total Claim Amount']

In [396]:
numericals = X._get_numeric_data()
numericals

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Coverage Transformed
0,4809.216960,48029,61,7.0,52,0.0,9,0
1,14947.917300,22139,100,34.0,31,0.0,2,0
2,22332.439460,49078,97,10.0,3,0.0,2,1
3,5035.035257,37405,63,8.0,99,3.0,4,0
4,4956.247235,87197,63,35.0,45,0.0,3,0
...,...,...,...,...,...,...,...,...
6812,3579.023825,28304,91,10.0,30,2.0,1,1
6813,2771.663013,59855,74,30.0,82,4.0,1,0
6814,5259.444853,61146,65,7.0,68,0.0,6,0
6815,23893.304100,39837,201,11.0,63,0.0,2,1


In [397]:
categoricals = X.select_dtypes('object')
categoricals

,State,Response,Education,EmploymentStatus,Gender,Location Code,Marital Status,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size
0,Arizona,No,College,Employed,M,Suburban,Married,Corporate,Offer3,Agent,Four-Door Car,Medsize
1,Washington,No,Bachelor,Employed,M,Suburban,Single,Personal,Offer3,Call Center,SUV,Medsize
2,Oregon,Yes,College,Employed,M,Suburban,Single,Corporate,Offer2,Branch,Four-Door Car,Medsize
3,California,No,Doctor,Employed,F,Urban,Married,Corporate,Offer2,Branch,Four-Door Car,Medsize
4,California,No,High School or Below,Employed,M,Urban,Single,Personal,Offer2,Branch,Two-Door Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...
6812,California,No,High School or Below,Employed,F,Suburban,Married,Personal,Offer2,Agent,Four-Door Car,Medsize
6813,Arizona,No,College,Employed,M,Suburban,Married,Personal,Offer2,Branch,Two-Door Car,Medsize
6814,Oregon,No,College,Employed,F,Urban,Married,Personal,Offer2,Branch,Four-Door Car,Medsize
6815,Arizona,No,Bachelor,Employed,F,Rural,Married,Corporate,Offer1,Web,Luxury SUV,Medsize


In [398]:
# one hot encoding
categoricals = pd.get_dummies(categoricals, drop_first=True)

In [399]:
# combining the processed numerical and categorical columns
all_features = pd.concat([numericals, categoricals], axis=1)
all_features

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Coverage Transformed,State_California,State_Nevada,State_Oregon,State_Washington,Response_Yes,Education_College,Education_Doctor,Education_High School or Below,Education_Master,EmploymentStatus_Employed,EmploymentStatus_Medical Leave,EmploymentStatus_Retired,Gender_M,Location Code_Suburban,Location Code_Urban,Marital Status_Married,Marital Status_Single,Policy_Personal,Policy_Special,Renew Offer Type_Offer2,Renew Offer Type_Offer3,Renew Offer Type_Offer4,Sales Channel_Branch,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small
0,4809.216960,48029,61,7.0,52,0.0,9,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,14947.917300,22139,100,34.0,31,0.0,2,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
2,22332.439460,49078,97,10.0,3,0.0,2,1,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
3,5035.035257,37405,63,8.0,99,3.0,4,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
4,4956.247235,87197,63,35.0,45,0.0,3,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6812,3579.023825,28304,91,10.0,30,2.0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
6813,2771.663013,59855,74,30.0,82,4.0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0
6814,5259.444853,61146,65,7.0,68,0.0,6,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
6815,23893.304100,39837,201,11.0,63,0.0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0


### Data splitting

In [400]:
X_train, X_test, y_train, y_test = train_test_split(all_features, y, test_size = 0.30, random_state = 100)

X_train = pd.DataFrame(X_train, columns=all_features.columns)
X_test = pd.DataFrame(X_test, columns=all_features.columns)

In [401]:
X_train.describe()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Coverage Transformed,State_California,State_Nevada,State_Oregon,State_Washington,Response_Yes,Education_College,Education_Doctor,Education_High School or Below,Education_Master,EmploymentStatus_Employed,EmploymentStatus_Medical Leave,EmploymentStatus_Retired,Gender_M,Location Code_Suburban,Location Code_Urban,Marital Status_Married,Marital Status_Single,Policy_Personal,Policy_Special,Renew Offer Type_Offer2,Renew Offer Type_Offer3,Renew Offer Type_Offer4,Sales Channel_Branch,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small
count,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000,4771.000000
mean,8161.831616,50586.191784,92.883882,15.119262,48.457556,0.392999,2.962272,0.474534,0.336407,0.097883,0.289457,0.088661,0.163907,0.296584,0.042130,0.264724,0.094949,0.839866,0.063718,0.038566,0.487948,0.539090,0.218822,0.666108,0.166003,0.740935,0.045064,0.351079,0.157409,0.117166,0.278558,0.190945,0.146510,0.017397,0.020960,0.189688,0.051981,0.209809,0.703626,0.196814
std,7007.620764,24217.250643,34.510460,10.079715,27.885255,0.919863,2.388881,0.652307,0.472530,0.297188,0.453558,0.284283,0.370230,0.456799,0.200906,0.441233,0.293175,0.366769,0.244276,0.192579,0.499907,0.498522,0.413491,0.471651,0.372122,0.438167,0.207466,0.477358,0.364224,0.321652,0.448336,0.393087,0.353654,0.130758,0.143265,0.392095,0.222011,0.407215,0.456706,0.397632
min,2119.894333,10037.000000,61.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4086.495083,28649.000000,68.000000,6.000000,24.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5821.128509,48259.000000,83.000000,14.000000,49.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,9035.671134,70260.500000,108.000000,23.000000,72.000000,0.000000,4.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,83325.381190,99981.000000,297.000000,35.000000,99.000000,5.000000,9.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Different regression models

In [402]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet,LinearRegression

In [403]:
# defining a function for different models
def regression_model(model_type):
    if model_type == 'OLS':
        model = LinearRegression()
    elif model_type == 'Lasso':
        alpha_input = float(input('Please input alpha: '))
        model = Lasso(alpha=alpha_input)
    elif model_type == 'Ridge':
        alpha_input = int(input('Please input alpha: '))
        model = Ridge(alpha=alpha_input)
    model.fit(X_train, y_train)
    print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [404]:
# OLS
regression_model('OLS')

LinearRegression: Train -> 0.8056991617101379, Test -> 0.7966645715048973


In [405]:
# Lasso
regression_model('Lasso')

Please input alpha: 0.05
Lasso: Train -> 0.8055614496712247, Test -> 0.7965939296330886


In [406]:
# Ridge
regression_model('Ridge')

Please input alpha: 100
Ridge: Train -> 0.7955431163285939, Test -> 0.7887023760629269


### Feature selection: P-Value

In [407]:
X = all_features
y = ca_df['Total Claim Amount']

X_added_constant = sm.add_constant(X)
X_added_constant

,const,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Coverage Transformed,State_California,State_Nevada,State_Oregon,State_Washington,Response_Yes,Education_College,Education_Doctor,Education_High School or Below,Education_Master,EmploymentStatus_Employed,EmploymentStatus_Medical Leave,EmploymentStatus_Retired,Gender_M,Location Code_Suburban,Location Code_Urban,Marital Status_Married,Marital Status_Single,Policy_Personal,Policy_Special,Renew Offer Type_Offer2,Renew Offer Type_Offer3,Renew Offer Type_Offer4,Sales Channel_Branch,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small
0,1.0,4809.216960,48029,61,7.0,52,0.0,9,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,1.0,14947.917300,22139,100,34.0,31,0.0,2,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
2,1.0,22332.439460,49078,97,10.0,3,0.0,2,1,0,0,1,0,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
3,1.0,5035.035257,37405,63,8.0,99,3.0,4,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
4,1.0,4956.247235,87197,63,35.0,45,0.0,3,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6812,1.0,3579.023825,28304,91,10.0,30,2.0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0
6813,1.0,2771.663013,59855,74,30.0,82,4.0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,1,0
6814,1.0,5259.444853,61146,65,7.0,68,0.0,6,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
6815,1.0,23893.304100,39837,201,11.0,63,0.0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0


In [408]:
model = sm.OLS(y,X_added_constant).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Total Claim Amount   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     692.5
Date:                Mon, 21 Feb 2022   Prob (F-statistic):               0.00
Time:                        21:40:04   Log-Likelihood:                -41839.
No. Observations:                6817   AIC:                         8.376e+04
Df Residuals:                    6776   BIC:                         8.404e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           -317.0980     16.402    -19.333      0.000    -349.250    -284.946
Customer Lifetime Value            0.0003      0.000      1.619      0.105   -7.35e-05       0.001
Income                            -0.0003   6.92e-05     -4.334      0.000      -0.000      -0.000
Monthly Premium Auto               5.0921      0.196     26.019      0.000       4.708       5.476
Months Since Last Claim            0.2018      0.136      1.483      0.138      -0.065       0.468
Months Since Policy Inception     -0.0157      0.049     -0.317      0.751      -0.112       0.081
Number of Open Complaints         -1.2491      1.511     -0.826      0.409      -4.212       1.714
Number of Policies                 0.6819      0.576      1.184      0.236      -0.447       1.811
Coverage Transformed             -20.0983      4.976     -4.039      0.000     -29.854     -10.343
State_California                   0.6510      3.922      0.166      0.868      -7.037       8.339
State_Nevada                      -0.8173      5.392     -0.152      0.880     -11.387       9.752
State_Oregon                      -1.7074      4.061     -0.420      0.674      -9.668       6.253
State_Washington                   6.1455      5.588      1.100      0.271      -4.809      17.100
Response_Yes                      -3.8106      4.118     -0.925      0.355     -11.883       4.262
Education_College                 -3.8737      3.552     -1.091      0.275     -10.836       3.088
Education_Doctor                 -13.2955      7.105     -1.871      0.061     -27.224       0.633
Education_High School or Below     1.4771      3.657      0.404      0.686      -5.692       8.647
Education_Master                  -3.0184      5.106     -0.591      0.554     -13.027       6.990
EmploymentStatus_Employed         -5.6236      6.350     -0.886      0.376     -18.071       6.824
EmploymentStatus_Medical Leave    14.7427      7.801      1.890      0.059      -0.550      30.035
EmploymentStatus_Retired         -12.5507      9.039     -1.389      0.165     -30.270       5.168
Gender_M                           8.3670      2.754      3.038      0.002       2.968      13.766
Location Code_Suburban           377.0231      3.645    103.442      0.000     369.878     384.168
Location Code_Urban              219.8590      4.015     54.759      0.000     211.988     227.730
Marital Status_Married            -6.1391      3.789     -1.620      0.105     -13.567       1.289
Marital Status_Single              4.0860      4.725      0.865      0.387      -5.176      13.348
Policy_Personal                    5.0589      3.343      1.513      0.130      -1.494      11.611
Policy_Special                    14.6327      7.293      2.006      0.045       0.337      28.929
Renew Offer Ty

In [409]:
# identifying columns that have a p-value higher than 0.05: 1st round
pvalues_df = pd.DataFrame(model.pvalues)
pvalues_df.columns=['P-Value']
drop_list = list(pvalues_df[pvalues_df['P-Value']>0.05].index)

In [410]:
# dropping insignificant features: 1st round
X_added_constant2 = X_added_constant.drop(drop_list, axis=1)
model2 = sm.OLS(y,X_added_constant2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Total Claim Amount   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     2120.
Date:                Mon, 21 Feb 2022   Prob (F-statistic):               0.00
Time:                        21:40:05   Log-Likelihood:                -41863.
No. Observations:                6817   AIC:                         8.375e+04
Df Residuals:                    6803   BIC:                         8.385e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     -314.9282     13.739    -22.923      0.000    -341.860    -287.996
Income                      -0.0003      6e-05     -5.830      0.000      -0.000      -0.000
Monthly Premium Auto         5.1279      0.195     26.313      0.000       4.746       5.510
Coverage Transformed       -20.7469      4.969     -4.175      0.000     -30.488     -11.006
Gender_M                     8.4704      2.742      3.089      0.002       3.095      13.846
Location Code_Suburban     378.4395      3.493    108.328      0.000     371.591     385.288
Location Code_Urban        219.6294      4.014     54.714      0.000     211.760     227.498
Policy_Special              10.9211      6.810      1.604      0.109      -2.429      24.271
Renew Offer Type_Offer2      8.0043      3.049      2.625      0.009       2.027      13.981
Renew Offer Type_Offer3      6.6813      3.955      1.689      0.091      -1.071      14.434
Vehicle Class_Luxury Car   -83.9872     27.956     -3.004      0.003    -138.790     -29.184
Vehicle Class_Luxury SUV   -76.7786     27.822     -2.760      0.006    -131.319     -22.238
Vehicle Class_SUV          -49.5203      9.205     -5.379      0.000     -67.566     -31.475
Vehicle Class_Sports Car   -66.2716     10.646     -6.225      0.000     -87.141     -45.402
==============================================================================
Omnibus:                     4165.029   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           157384.948
Skew:                           2.340   Prob(JB):                         0.00
Kurtosis:                      26.069   Cond. No.                     1.73e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.73e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [411]:
# identifying columns that have a p-value higher than 0.05: 2nd round
pvalues_df2 = pd.DataFrame(model2.pvalues)
pvalues_df2.columns=['P-Value']
drop_list2 = list(pvalues_df2[pvalues_df2['P-Value']>0.05].index)

In [412]:
# dropping insignificant features: 2nd round
X_added_constant3 = X_added_constant2.drop(drop_list2, axis=1)
model3 = sm.OLS(y,X_added_constant3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Total Claim Amount   R-squared:                       0.802
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     2504.
Date:                Mon, 21 Feb 2022   Prob (F-statistic):               0.00
Time:                        21:40:06   Log-Likelihood:                -41866.
No. Observations:                6817   AIC:                         8.376e+04
Df Residuals:                    6805   BIC:                         8.384e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     -312.5594     13.693    -22.827      0.000    -339.402    -285.717
Income                      -0.0003   5.99e-05     -5.751      0.000      -0.000      -0.000
Monthly Premium Auto         5.1213      0.195     26.276      0.000       4.739       5.503
Coverage Transformed       -20.6919      4.970     -4.163      0.000     -30.435     -10.948
Gender_M                     8.4762      2.743      3.090      0.002       3.099      13.853
Location Code_Suburban     378.3527      3.494    108.291      0.000     371.504     385.202
Location Code_Urban        219.7773      4.015     54.744      0.000     211.907     227.647
Renew Offer Type_Offer2      6.3785      2.887      2.209      0.027       0.718      12.039
Vehicle Class_Luxury Car   -83.0673     27.959     -2.971      0.003    -137.875     -28.259
Vehicle Class_Luxury SUV   -76.1858     27.828     -2.738      0.006    -130.737     -21.635
Vehicle Class_SUV          -49.3914      9.208     -5.364      0.000     -67.441     -31.342
Vehicle Class_Sports Car   -66.0864     10.649     -6.206      0.000     -86.961     -45.212
==============================================================================
Omnibus:                     4169.389   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           157921.192
Skew:                           2.343   Prob(JB):                         0.00
Kurtosis:                      26.109   Cond. No.                     1.73e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.73e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Variance threshold method

In [424]:
X = ca_df.drop('Total Claim Amount', axis=1)
y = ca_df['Total Claim Amount']

from sklearn.feature_selection import VarianceThreshold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

print("Initial number of columns: ",X_train.shape,'\n')

selector = VarianceThreshold(threshold=1000)
selector.fit(X_train)

kept_features_indexes = selector.get_support(indices = True)
kept_features = list(X_train.iloc[:,kept_features_indexes].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final number of columns: ",X_train.shape)
print()
X_train

Initial number of columns:  (4771, 8) 

Final number of columns:  (4771, 3)



,Customer Lifetime Value,Income,Monthly Premium Auto
0,7164.395543,46623.0,91.0
1,5427.174517,27677.0,72.0
2,3433.824080,85367.0,85.0
3,5528.664986,16042.0,73.0
4,2869.997625,70974.0,71.0
...,...,...,...
4766,7870.360084,75740.0,98.0
4767,35862.993240,26376.0,121.0
4768,4076.634665,29549.0,104.0
4769,4219.024120,62396.0,107.0


### Recursive feature elimination

In [426]:
X = ca_df.drop('Total Claim Amount', axis=1)
y = ca_df['Total Claim Amount']

from sklearn.feature_selection import RFE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

X_train = X_train.select_dtypes(include=np.number)
X_test  = X_test.select_dtypes(include=np.number)

lm = LinearRegression()

selector = RFE(lm, n_features_to_select= 4, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train, y_train)

kept_features_indexes = selector.get_support(indices = True)
kept_features = list(X_train.iloc[:,kept_features_indexes].columns)

X_train = selector.transform(X_train)
X_test  = selector.transform(X_test)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final selected features: ")
display(X_train)

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Final selected features: 


,Monthly Premium Auto,Number of Open Complaints,Number of Policies,Coverage Transformed
0,91.0,0.0,3.0,1.0
1,72.0,1.0,3.0,0.0
2,85.0,0.0,1.0,1.0
3,73.0,0.0,9.0,0.0
4,71.0,0.0,1.0,0.0
...,...,...,...,...
4766,98.0,0.0,3.0,1.0
4767,121.0,0.0,2.0,1.0
4768,104.0,0.0,1.0,0.0
4769,107.0,0.0,1.0,0.0
